In [4]:
# FINAL VERSION 2025-03-28
# This script 
# 1. prompts user for concert pitch A4 in Hz
# 2. prompts user for two frequencies in Hz
# 3. determines the interval between those frequencies in Cents
# 4. compares frequencies and interval to custom tuning table in semicolon-delimited CSV file. 
# 5. returns note names ±50 Cts. and interval in semitones ±50 Cts for each custom tuning, relative to concert pitch A4.


import csv
import numpy as np

def load_tuning_table(csv_file):
    # Load tuning deviations from a semicolon-delimited CSV file.
    tunings = {}
    with open(csv_file, newline='', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=';')
        headers = next(reader)[1:]  # Skip first column (tuning names), keep note names
        for row in reader:
            tuning_name = row[0]
            deviations = list(map(float, row[1:]))
            if len(deviations) == 12:
                tunings[tuning_name] = deviations
            else:
                print(f"Warning: {tuning_name} has {len(deviations)} deviations (expected 12). Skipping.")
    return headers, tunings

def frequency_to_midi(frequency, a4):
    # Convert a frequency to the closest MIDI note number.
    return 69 + 12 * np.log2(frequency / a4)

def midi_to_note(midi_number, note_names):
    # Convert a MIDI number to a note name with octave.
    note_index = int(midi_number) % 12
    octave = (int(midi_number) // 12) - 1
    return f"{note_names[note_index]}{octave}"

def find_closest_note(frequency, note_names, tunings, a4):
    # Find the closest note in each tuning, ensuring deviations stay within ±50 cents.
    midi_number = frequency_to_midi(frequency, a4)
    closest_midi = round(midi_number)
    deviation_from_standard = (midi_number - closest_midi) * 100
    
    results = {}
    for tuning, deviations in tunings.items():
        note_index = closest_midi % 12
        tuning_deviation = deviations[note_index]
        total_deviation = deviation_from_standard - tuning_deviation

        # Ensure deviations stay within ±50 cents
        if total_deviation > 50:
            closest_midi += 1
            total_deviation -= 100
        elif total_deviation < -50:
            closest_midi -= 1
            total_deviation += 100
        
        note_name = midi_to_note(closest_midi, note_names)
        results[tuning] = (note_name, tuning_deviation, total_deviation)
    
    return results

def calculate_interval(freq1, freq2, tunedev1, tunedev2):
    # Calculate interval between two frequencies in cents and semitones, for custom tuning.
    if freq2 >= freq1:
        equal_interval_cents = 1200 * np.log2(freq2 / freq1)
        adjusted_interval_cents = equal_interval_cents - tunedev1 + tunedev2 # adjust for tuning deviations
    else: 
        equal_interval_cents = 1200 * np.log2(freq1 / freq2)
        adjusted_interval_cents = equal_interval_cents - tunedev2 + tunedev1 # adjust for tuning deviations
    
    # break up interval into semitones and cents
    semitones = int(round(adjusted_interval_cents / 100))
    remaining_cents = adjusted_interval_cents - (semitones * 100)

    # Ensure remaining cents is within ±50 cents
    if remaining_cents > 50:
        semitones += 1
        remaining_cents -= 100
    elif remaining_cents < -50:
        semitones -= 1
        remaining_cents += 100

    return semitones, remaining_cents

def get_frequency_input(prompt):
    # Prompt user input for a valid frequency.
    while True:
        try:
            value = float(input(prompt))
            if value > 0:
                return value
            else:
                print("Frequency must be greater than 0.")
        except ValueError:
            print("Invalid input. Please enter a number.")

def main():
    csv_file = 'example_temperaments.csv' 
    # example data sourced from http://www.instrument-tuner.com/temperaments.html on 2024/08/26
    
    note_names, tunings = load_tuning_table(csv_file)


    a4 = get_frequency_input("Enter concert pitch in Hz: ")
    freq1 = get_frequency_input("Enter the first frequency in Hz: ")
    freq2 = get_frequency_input("Enter the second frequency in Hz: ")
    print()
    
    print(f"Results for each tuning, at concert pitch A4 = {a4}: ")
    results1 = find_closest_note(freq1, note_names, tunings, a4)
    results2 = find_closest_note(freq2, note_names, tunings, a4)

     
    for tuning in tunings:
        note1, tunedev1, dev1 = results1[tuning]
        note2, tunedev2, dev2 = results2[tuning]
    
        semitones, remaining_cents = calculate_interval(freq1, freq2, tunedev1, tunedev2)

        remaining_cents_abs = abs(remaining_cents) # Ensure returned cent interval is positive
        print(f"{tuning}: \n{note1} ({dev1:+.2f} cents) to {note2} ({dev2:+.2f} cents), \nInterval: {semitones} semitones {remaining_cents_abs:.2f} cents \n")

if __name__ == "__main__":
    main()

Enter concert pitch in Hz:  438
Enter the first frequency in Hz:  asölgkh


Invalid input. Please enter a number.


Enter the first frequency in Hz:  245
Enter the second frequency in Hz:  .nn


Invalid input. Please enter a number.


Enter the second frequency in Hz:  3476



Results for each tuning, at concert pitch A4 = 438.0: 
Equal Tempered, Perfect Octave: 
B3 (-5.78 cents) to A7 (-13.89 cents), 
Interval: 46 semitones 8.11 cents 

Equal Tempered, Perfect Fourth: 
B3 (-5.00 cents) to A7 (-13.89 cents), 
Interval: 46 semitones 7.33 cents 

Equal Tempered, Perfect Fifth: 
B3 (-6.34 cents) to A7 (-13.89 cents), 
Interval: 46 semitones 8.67 cents 

Equal Tempered, Streched (1.0 Cent): 
B3 (-5.95 cents) to A7 (-13.89 cents), 
Interval: 46 semitones 8.28 cents 

Equal Tempered, Streched (1.25 Cent): 
B3 (-5.99 cents) to A7 (-13.89 cents), 
Interval: 46 semitones 8.32 cents 

Equal Tempered, Streched (1.5 Cent): 
B3 (-6.03 cents) to A7 (-13.89 cents), 
Interval: 46 semitones 8.36 cents 

Just Tempered (Schugk): 
B3 (-9.69 cents) to A7 (-13.89 cents), 
Interval: 46 semitones 12.02 cents 

Just Tempered (Barbour): 
B3 (-9.69 cents) to A7 (-13.89 cents), 
Interval: 46 semitones 12.02 cents 

Naturally harmonious (Thirds): 
B3 (-9.69 cents) to A7 (-13.89 cents),